# 第13章 RAG

## 13.2 基本的な RAG のシステムの実装

### 13.2.2 LangChain で LLM と文埋め込みモデルを使う

#### 環境の準備

In [ ]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

#### LangChain で LLM を使う

In [ ]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 128,
    "do_sample": False,
    "temperature": None,
    "top_p": None,
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config,
)

# パイプラインからLangChainのLLMコンポーネントを作成
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
from pprint import pprint

# モデルに入力する会話データ
llm_prompt_messages = [
    {"role": "user", "content": "四国地方で一番高い山は？"},
]

# 会話データにチャットテンプレートを適用し、内容を確認
llm_prompt_text = tokenizer.apply_chat_template(
    llm_prompt_messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(llm_prompt_text)

In [ ]:
# LLMへの入力を実行し、結果を確認
llm_output_message = llm.invoke(llm_prompt_text)
print(llm_output_message)

#### Chat Model コンポーネントの利用

In [ ]:
from langchain_huggingface import ChatHuggingFace

# LLMコンポーネントからChat Modelコンポーネントを作成
chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Chat Modelに入力する会話データ
chat_messages = [HumanMessage(content="四国地方で一番高い山は？")]

# Chat Modelによるチャットテンプレート適用後の入力文字列を確認
chat_prompt = chat_model._to_chat_prompt(chat_messages)
print(chat_prompt)

In [ ]:
# Chat Modelに会話データを入力し、出力を確認
chat_output_message = chat_model.invoke(chat_messages)
pprint(chat_output_message)

In [ ]:
# Chat Modelが出力したテキストからモデルの応答部分のみを抽出
response_text = chat_output_message.content[len(chat_prompt) :]
print(response_text)

#### Chain を構築する

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからプロンプトを構築するPrompt Templateを作成
prompt_template = ChatPromptTemplate.from_messages(
    [("user", "{query}")]
)

# Prompt Templateを実行し、結果を確認
prompt_template_output = prompt_template.invoke(
    {"query": "四国地方で一番高い山は？"}
)
pprint(prompt_template_output)

In [ ]:
# Prompt TemplateとChat Modelを連結したChainを作成
chain = prompt_template | chat_model

# Chainを実行し、結果を確認
chain_output = chain.invoke({"query": "四国地方で一番高い山は？"})
pprint(chain_output)

In [ ]:
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda

def chat_model_resp_only_func(
    chat_prompt_value: ChatPromptValue,
) -> str:
    """chat_modelにchat_prompt_valueを入力し、
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt) :]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

# Prompt TemplateとRunnableを連結したChainを作成
chain_resp_only = prompt_template | chat_model_resp_only

# Chainを実行し、結果を確認
chain_resp_only_output = chain_resp_only.invoke(
    {"query": "四国地方で一番高い山は？"}
)
print(chain_resp_only_output)

#### LangChain で文埋め込みモデルを使う


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face Hubにおけるモデル名を指定
embedding_model_name = "BAAI/bge-m3"

# モデル名からEmbedding Modelを初期化
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

In [ ]:
sample_texts = [
    "日本で一番高い山は何ですか？",
    "日本で一番高い山は富士山です。",
]

# 二つのテキストに対して文埋め込みを実行し、結果を確認
sample_embeddings = embedding_model.embed_documents(sample_texts)
print(sample_embeddings)

In [ ]:
# 二つのテキストの文埋め込みから類似度を計算
similarity = torch.nn.functional.cosine_similarity(
    torch.tensor([sample_embeddings[0]]),
    torch.tensor([sample_embeddings[1]]),
)
print(similarity)

### 13.2.3 LangChain で RAG を実装する

#### データストアの構築



In [ ]:
# 検索対象の文書集合のファイルをダウンロード
!wget \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

In [ ]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))

In [ ]:
# 読み込まれた文書の内容を確認
pprint(documents[0])

In [ ]:
# 読み込まれた文書の長さ（文字数）を確認
print(len(documents[0].page_content))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

In [ ]:
# 分割後の文書の内容を確認
pprint(split_documents[0])
pprint(split_documents[1])

In [ ]:
# 分割後の文書の長さ（文字数）を確認
print(len(split_documents[0].page_content))
print(len(split_documents[1].page_content))

#### ベクトルインデックスの作成

In [ ]:
from langchain_community.vectorstores import FAISS

# 分割後の文書と文埋め込みモデルを用いて、Faissのベクトルインデックスを作成
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# ベクトルインデックスに登録された文書数を確認
print(vectorstore.index.ntotal)

#### Retriever コンポーネントの作成

In [ ]:
# ベクトルインデックスを元に文書の検索を行うRetrieverを初期化
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 文書の検索を実行
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

# 検索された文書を確認
pprint(retrieved_documents)

#### RAG の Chain の構築

In [ ]:
# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "以下の文書の内容を参考にして、質問に答えてください。\n\n"
    "---\n{context}\n---\n\n質問: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [ ]:
from langchain_core.documents import Document

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

# 定義した関数の処理を行うRunnableを作成
format_documents = RunnableLambda(format_documents_func)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAGの一連の処理を行うChainを作成
rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [ ]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)